In [3]:
!pip install numpy
!pip install matplotlib
!pip install tensorflow
!pip install scipy
!pip install scikit-image
!pip install pandas
!pip install keras
!pip install scikit-learn

In [1]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from scipy import ndimage
from skimage.measure import regionprops
from skimage import io
from skimage.filters import threshold_otsu   # For finding the threshold for grayscale to binary conversion
import tensorflow as tf
import pandas as pd
import numpy as np
from time import time
import keras
from tensorflow.python.framework import ops
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

## Path defined

In [2]:
genuine_image_paths =r"C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\real"
forged_image_paths = r"C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\forged"

## Preprocessing the image

In [3]:
def rgbgrey(img):
    # Converts rgb to grayscale
    greyimg = np.zeros((img.shape[0], img.shape[1]))
    for row in range(len(img)):
        for col in range(len(img[row])):
            greyimg[row][col] = np.average(img[row][col])
    return greyimg

In [4]:
def greybin(img):
    # Converts grayscale to binary
    blur_radius = 0.8
    img = ndimage.gaussian_filter(img, blur_radius)  # to remove small components or noise
#     img = ndimage.binary_erosion(img).astype(img.dtype)
    thres = threshold_otsu(img)
    binimg = img > thres
    binimg = np.logical_not(binimg)
    return binimg

In [5]:
def preproc(path, img=None, display=True):
    if img is None:
        img = mpimg.imread(path)
    if display:
        plt.imshow(img)
        plt.show()
    grey = rgbgrey(img) #rgb to grey
    if display:
        plt.imshow(grey, cmap = matplotlib.cm.Greys_r)
        plt.show()
    binimg = greybin(grey) #grey to binary
    if display:
        plt.imshow(binimg, cmap = matplotlib.cm.Greys_r)
        plt.show()
    r, c = np.where(binimg==1)
    # Now we will make a bounding box with the boundary as the position of pixels on extreme.
    # Thus we will get a cropped image with only the signature part.
    signimg = binimg[r.min(): r.max(), c.min(): c.max()]
    if display:
        plt.imshow(signimg, cmap = matplotlib.cm.Greys_r)
        plt.show()
    return signimg

## Feature Extraction


In [6]:
def Ratio(img):
    a = 0
    for row in range(len(img)):
        for col in range(len(img[0])):
            if img[row][col]==True:
                a = a+1
    total = img.shape[0] * img.shape[1]
    return a/total

In [7]:
def Centroid(img):
    numOfWhites = 0
    a = np.array([0,0])
    for row in range(len(img)):
        for col in range(len(img[0])):
            if img[row][col]==True:
                b = np.array([row,col])
                a = np.add(a,b)
                numOfWhites += 1
    rowcols = np.array([img.shape[0], img.shape[1]])
    centroid = a/numOfWhites
    centroid = centroid/rowcols
    return centroid[0], centroid[1]

In [8]:
def EccentricitySolidity(img):
    r = regionprops(img.astype("int8"))
    return r[0].eccentricity, r[0].solidity

In [9]:
def SkewKurtosis(img):
    h,w = img.shape
    x = range(w)  # cols value
    y = range(h)  # rows value
    #calculate projections along the x and y axes
    xp = np.sum(img,axis=0)
    yp = np.sum(img,axis=1)
    #centroid
    cx = np.sum(x*xp)/np.sum(xp)
    cy = np.sum(y*yp)/np.sum(yp)
    #standard deviation
    x2 = (x-cx)**2
    y2 = (y-cy)**2
    sx = np.sqrt(np.sum(x2*xp)/np.sum(img))
    sy = np.sqrt(np.sum(y2*yp)/np.sum(img))
    
    #skewness
    x3 = (x-cx)**3
    y3 = (y-cy)**3
    skewx = np.sum(xp*x3)/(np.sum(img) * sx**3)
    skewy = np.sum(yp*y3)/(np.sum(img) * sy**3)

    #Kurtosis
    x4 = (x-cx)**4
    y4 = (y-cy)**4
    # 3 is subtracted to calculate relative to the normal distribution
    kurtx = np.sum(xp*x4)/(np.sum(img) * sx**4) - 3
    kurty = np.sum(yp*y4)/(np.sum(img) * sy**4) - 3

    return (skewx , skewy), (kurtx, kurty)

In [10]:
def getFeatures(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    img = preproc(path, display=display)
    ratio = Ratio(img)
    centroid = Centroid(img)
    eccentricity, solidity = EccentricitySolidity(img)
    skewness, kurtosis = SkewKurtosis(img)
    retVal = (ratio, centroid, eccentricity, solidity, skewness, kurtosis)
    return retVal

In [11]:
def getCSVFeatures(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    temp = getFeatures(path, display=display)
    features = (temp[0], temp[1][0], temp[1][1], temp[2], temp[3], temp[4][0], temp[4][1], temp[5][0], temp[5][1])
    return features

## Saving the features

In [12]:
def makeCSV():
    if not(os.path.exists(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\Features')):
        os.mkdir(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\Features')
        print('New folder "Features" created')
    if not(os.path.exists(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\Features/Training')):
        os.mkdir(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\Features/Training')
        print('New folder "Features/Training" created')
    if not(os.path.exists(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\Features/Testing')):
        os.mkdir(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\Features/Testing')
        print('New folder "Features/Testing" created')
    # genuine signatures path
    gpath = genuine_image_paths
    # forged signatures path
    fpath = forged_image_paths
    for person in range(1,13):
        per = ('00'+str(person))[-3:]
        print('Saving features for person id-',per)
        
        with open(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\\Features\\Training/training_'+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            # Training set
            for i in range(0,3):
                source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',1\n')
            for i in range(0,3):
                source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',0\n')
        
        with open(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\\Features\\Testing/testing_'+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            # Testing set
            for i in range(3, 5):
                source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',1\n')
            for i in range(3,5):
                source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',0\n')

In [13]:
import os

def addcsv():
    per = input("Enter person's id: ")
    gpath = genuine_image_paths
    fpath = forged_image_paths    
    with open(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\\Features\\Training/training_'+per+'.csv', 'w') as handle:
        handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
        # Training set
        for i in range(0, 3):
            source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
            features = getCSVFeatures(path=source)
            handle.write(','.join(map(str, features))+',1\n')
        for i in range(0, 3):
            source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
            features = getCSVFeatures(path=source)
            handle.write(','.join(map(str, features))+',0\n')
    with open(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\\Features\\Testing/testing_'+per+'.csv', 'w') as handle:
        handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
        # Testing set
        for i in range(3, 5):
            source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
            features = getCSVFeatures(path=source)
            handle.write(','.join(map(str, features))+',1\n')
        for i in range(3, 5):
            source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
            features = getCSVFeatures(path=source)
            handle.write(','.join(map(str, features))+',0\n')
    print('Saving features for person id-',per)


In [14]:
# addcsv()

In [15]:
makeCSV()

Saving features for person id- 001
Saving features for person id- 002
Saving features for person id- 003
Saving features for person id- 004
Saving features for person id- 005
Saving features for person id- 006
Saving features for person id- 007
Saving features for person id- 008
Saving features for person id- 009
Saving features for person id- 010
Saving features for person id- 011
Saving features for person id- 012


# TF Model 

In [16]:
def testing(path):
    feature = getCSVFeatures(path)
    if not(os.path.exists(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main/TestFeatures')):
        os.mkdir(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main/TestFeatures')
    with open(r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main/TestFeatures/testcsv.csv', 'w') as handle:
        handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y\n')
        handle.write(','.join(map(str, feature))+'\n')

In [ ]:
n_input = 9
train_person_id = input("Enter person's id: ")
test_image_path = input("Enter path of signature image: ")
train_path = r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main\\Features\\Training/training_'+train_person_id+'.csv'
testing(test_image_path)
test_path = r'C:\Users\Charan\OneDrive\Desktop\Signature-Forgery-Detection-main/TestFeatures/testcsv.csv'
def readCSV(train_path, test_path):
    # Reading train data
    train_df = pd.read_csv(train_path, usecols=range(n_input))
    train_labels_df = pd.read_csv(train_path, usecols=[n_input])
    train_data = np.array(train_df.values, dtype=np.float32)
    train_labels = keras.utils.to_categorical(train_labels_df.values.flatten(), num_classes=2)
    
    # Reading test data
    test_df = pd.read_csv(test_path, usecols=range(n_input))
    test_data = np.array(test_df.values, dtype=np.float32)
    
    return train_data, train_labels, test_data

train_data, train_labels, test_data = readCSV(train_path, test_path)

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# Define the model architecture using Keras Sequential API
model = keras.Sequential([
    keras.layers.Dense(7, activation='tanh', input_shape=(n_input,)),
    keras.layers.Dense(10, activation='tanh'),
    keras.layers.Dense(30, activation='tanh'),
    keras.layers.Dense(2, activation='softmax')
])

model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, epochs=1000, batch_size=32, verbose=1)

# Evaluate the model on training and validation data
train_loss, train_acc = model.evaluate(train_data, train_labels, verbose=0)
val_loss, val_acc = model.evaluate(val_data, val_labels, verbose=0)
print("Training accuracy:", train_acc)
print("Validation accuracy:", val_acc)

# Predict on the test data
test_predictions = model.predict(test_data)
predicted_class = np.argmax(test_predictions[0])
if predicted_class == 1:
    print("Genuine Image")
else:
    print("Forged Image")

Enter person's id:  001
Enter path of signature image:  C:\Users\Charan\OneDrive\Desktop\DigitalSignatureVerificationMP2-master\forged\021001_001.png


C:\Users\Charan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 7)                   │              70 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 30)                  │             330 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │              62 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 542 (2.12 KB)

 Trainable params: 542 (2.12 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2500 - loss: 0.6788
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 1.0000 - loss: 0.6607
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 0.6439
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 1.0000 - loss: 0.6283
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 1.0000 - loss: 0.6138
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.7500 - loss: 0.6005
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7500 - loss: 0.5882
Epoch 8/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.7500 - loss: 0.5770
Epoch 9/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.7500 - loss: 0.5667
Epoch 10/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.7500 - loss: 0.5573
Epoch 11/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.7500 - loss: 0.5488
Epoch 12/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0